# Heuristicas

In [56]:
import utils
import pandas as pd
import seaborn as sns
from typing import List
import simulator
import numpy as np
import copy

DATA = "../data/sim.dat"
HEUR = "../data/sim.heuristic.dat"
POWERS = "../data/sim_powers.tsv"

In [2]:
def read_powers() -> pd.DataFrame:
    return pd.read_csv(
        POWERS,
        sep=' ',
        header=None,
        names=["team", "power"],
    )

def read_sim_ranking(path: str) -> pd.DataFrame:
    powers = read_powers()
    ranking = utils.read_ranking(path)
    ranking["power"] = (ranking.index + 1).map(powers.set_index("team")["power"])
    ranking["team"] = (ranking.index+1)
    return ranking

In [3]:
def games2cmm(games: List[simulator.Game], teams):
    b = np.zeros((1,teams))[0]
    cmm = np.zeros(shape=(teams, teams))
    
    for game in games:
        cmm[game.team1-1][game.team2-1]-=1
        cmm[game.team2-1][game.team1-1]-=1
        cmm[game.team2-1][game.team2-1]+=1
        cmm[game.team1-1][game.team1-1]+=1
        if game.goals1 > game.goals2:
            b[game.team1-1]+=1
            b[game.team2-1]-=1
        else:
            b[game.team1-1]-=1
            b[game.team2-1]+=1

    for i in range(0,teams):
        cmm[i][i] += 2
        b[i] /= 2
        b[i] += 1
            
    return np.linalg.solve(cmm,b)

In [106]:
PLAYER = 3
def rank_and_worst(player, games, teams) -> (int, int):
    ratings = games2cmm(games, teams)
    rank = sum(ratings > ratings[player-1])
    worst_rating = ratings[0]
    worst_player_index = 0
    
    #print("r:",ratings[PLAYER-1])
    for i in range(len(ratings)):
        if ratings[i] < worst_rating:
            worst_player_index = i
    
    return rank,worst_player_index +1

def lose_against(player1, player2, games):
    aux = copy.deepcopy(games)
    
    for i in range(len(aux)):
        if ((aux[i].team1 == player1 and
            aux[i].team2 == player2 and 
            aux[i].goals1 > aux[i].goals2) or 
            (aux[i].team1 == player2 and
            aux[i].team2 == player1 and
            aux[i].goals2 > aux[i].goals1)):
            
            aux[i].goals1, aux[i].goals2 = aux[i].goals2, aux[i].goals1
            break
            
    return aux

def min_cmm(games: List[simulator.Game], teams, selected) -> List[simulator.Game]:
    old_games = games.copy()
    current_games = games.copy()
    #print("first:",old_games)
    for i in range(len(current_games)): #Cambiar
        old_games = current_games.copy()
        rank,worst_player = rank_and_worst(selected, current_games, len(teams))

        #for g in current_games:
        #    if (g.team1 == PLAYER and g.team2 == worst_player or
        #       g.team2 == PLAYER and g.team1 == worst_player):
        #        print(g)
        
        
        current_games = lose_against(selected, worst_player, old_games)
        
        #print("Third:",old_games)

        #print(old_games)
        #print("Fourth:",current_games)
        
        #print("-----------")
        #for g in current_games:
        #    if (g.team1 == PLAYER and g.team2 == worst_player or
        #       g.team2== PLAYER and g.team1 == worst_player):
        #        print(g)
                
        new_ratings = games2cmm(current_games, len(teams))
        new_rank = sum(new_ratings > new_ratings[selected-1])
        if (new_rank > rank):
            #print("old_games:",old_games)
            #print("current:",current_games)
            break
    return old_games

In [107]:
TEAM = 20
ITER = 500
ratings = []

#for i in range(ITER):
#if i % 100 == 0: print(i)

#simulator.simulate(games_against_each=1,goals_per_game=500,team_count=5)
#utils.run(DATA, utils.METHOD_CMM)
#df1 = read_sim_ranking(DATA+utils.METHOD_CMM+utils.OUT)
#df = utils.analyze_data(DATA)
#df

In [108]:
#df1

In [113]:
simulator.simulate(games_against_each=1,goals_per_game=500,team_count=4,heuristic=min_cmm)
utils.run(HEUR, utils.METHOD_CMM)
#utils.run(DATA, utils.METHOD_CMM)
#rankings_data = utils.analyze_data(DATA)
rakings_heur = utils.analyze_data(HEUR)
#ratings_data = read_sim_ranking(DATA+utils.METHOD_CMM+utils.OUT)
ratings_heur = read_sim_ranking(HEUR+utils.METHOD_CMM+utils.OUT)

#df = read_sim_ranking(HEUR+utils.METHOD_CMM+utils.OUT).sort_values("rating", ascending=False)
#df

In [114]:
ratings_heur

,rating,power,team
0,0.416667,26,1
1,0.416667,51,2
2,0.416667,76,3
3,0.750000,101,4


In [115]:
rakings_heur

,team,share,total,wins,participation
0,1.0,1.0,3.0,1.0,1.0
1,2.0,1.0,3.0,1.0,1.0
2,3.0,1.0,3.0,1.0,1.0
3,4.0,1.0,3.0,3.0,1.0


In [10]:
ratings_data

NameError: name 'ratings_data' is not defined

In [ ]:
ratings_heur